In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle

from time import time
from utils.df_loader import load_german_df
from utils.preprocessing import remove_missing_values, min_max_scale_numerical, inverse_dummy
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from alibi.explainers import CounterFactualProto, CounterFactual
from alibi_cf.utils import get_cat_vars_dict

pd.options.mode.chained_assignment = None 

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False


seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)


TF version:  2.4.0-rc0
Eager execution enabled:  True


In [2]:
df, feature_names, numerical_cols, categorical_cols, columns_type, target_name, possible_outcomes = load_german_df()

In [3]:
scaled_df, scaler = min_max_scale_numerical(df, numerical_cols)

In [4]:
scaled_df.head(5)

,default,account_check_status,duration_in_month,credit_history,purpose,credit_amount,savings,present_emp_since,installment_as_income_perc,personal_status_sex,...,present_res_since,property,age,other_installment_plans,housing,credits_this_bank,job,people_under_maintenance,telephone,foreign_worker
0,N,< 0 DM,0.029412,critical account/ other credits existing (not ...,domestic appliances,0.050567,unknown/ no savings account,.. >= 7 years,1.000000,male : single,...,1.000000,real estate,0.857143,none,own,0.333333,skilled employee / official,0.0,"yes, registered under the customers name",yes
1,Y,0 <= ... < 200 DM,0.647059,existing credits paid back duly till now,domestic appliances,0.313690,... < 100 DM,1 <= ... < 4 years,0.333333,female : divorced/separated/married,...,0.333333,real estate,0.053571,none,own,0.000000,skilled employee / official,0.0,none,yes
2,N,no checking account,0.117647,critical account/ other credits existing (not ...,(vacation - does not exist?),0.101574,... < 100 DM,4 <= ... < 7 years,0.333333,male : single,...,0.666667,real estate,0.535714,none,own,0.000000,unskilled - resident,1.0,none,yes
3,N,< 0 DM,0.558824,existing credits paid back duly till now,radio/television,0.419941,... < 100 DM,4 <= ... < 7 years,0.333333,male : single,...,1.000000,if not A121 : building society savings agreeme...,0.464286,none,for free,0.000000,skilled employee / official,1.0,none,yes
4,Y,< 0 DM,0.294118,delay in paying off in the past,car (new),0.254209,... < 100 DM,1 <= ... < 4 years,0.666667,male : single,...,1.000000,unknown / no property,0.607143,none,for free,0.333333,skilled employee / official,1.0,none,yes


In [5]:
dummy_df = pd.get_dummies(scaled_df, columns=  [ col for col in categorical_cols if col != target_name])

In [6]:
### We should have this amount of input features.
sum([len(scaled_df[col].unique()) for col in categorical_cols if col != target_name]) + len(numerical_cols)

61

In [7]:
# label_enconded_df, encoder_dict = label_encode(scaled_df, categorical_cols)

In [8]:
cat_to_ohe_cat = {}
for c_col in categorical_cols:
    if c_col != target_name:
        cat_to_ohe_cat[c_col] = [ ohe_col for ohe_col in dummy_df.columns if ohe_col.startswith(c_col) and ohe_col != target_name]

In [9]:
ohe_feature_names = [ col for col in dummy_df.columns if col != target_name]

In [10]:
dummy_df.head(5)

,default,duration_in_month,credit_amount,installment_as_income_perc,present_res_since,age,credits_this_bank,people_under_maintenance,account_check_status_0 <= ... < 200 DM,account_check_status_< 0 DM,...,housing_own,housing_rent,job_management/ self-employed/ highly qualified employee/ officer,job_skilled employee / official,job_unemployed/ unskilled - non-resident,job_unskilled - resident,telephone_none,"telephone_yes, registered under the customers name",foreign_worker_no,foreign_worker_yes
0,N,0.029412,0.050567,1.000000,1.000000,0.857143,0.333333,0.0,0,1,...,1,0,0,1,0,0,0,1,0,1
1,Y,0.647059,0.313690,0.333333,0.333333,0.053571,0.000000,0.0,1,0,...,1,0,0,1,0,0,1,0,0,1
2,N,0.117647,0.101574,0.333333,0.666667,0.535714,0.000000,1.0,0,0,...,1,0,0,0,0,1,1,0,0,1
3,N,0.558824,0.419941,0.333333,1.000000,0.464286,0.000000,1.0,0,1,...,0,0,0,1,0,0,1,0,0,1
4,Y,0.294118,0.254209,0.666667,1.000000,0.607143,0.333333,1.0,0,1,...,0,0,0,1,0,0,1,0,0,1


In [11]:
# def inverse_dummy(dummy_df, cat_to_ohe_cat):
#     not_dummy_df = dummy_df.copy(deep=True)
#     for k in cat_to_ohe_cat.keys():
#         not_dummy_df[k] = dummy_df[cat_to_ohe_cat[k]].idxmax(axis=1)
#         not_dummy_df[k] = not_dummy_df[k].apply(lambda x: x.replace(f'{k}_',""))
#         not_dummy_df.drop(cat_to_ohe_cat[k], axis=1, inplace=True)
#     return not_dummy_df

In [12]:
inverse_dummy(dummy_df, cat_to_ohe_cat).head(5)

,default,duration_in_month,credit_amount,installment_as_income_perc,present_res_since,age,credits_this_bank,people_under_maintenance,account_check_status,credit_history,...,savings,present_emp_since,personal_status_sex,other_debtors,property,other_installment_plans,housing,job,telephone,foreign_worker
0,N,0.029412,0.050567,1.000000,1.000000,0.857143,0.333333,0.0,< 0 DM,critical account/ other credits existing (not ...,...,unknown/ no savings account,.. >= 7 years,male : single,none,real estate,none,own,skilled employee / official,"yes, registered under the customers name",yes
1,Y,0.647059,0.313690,0.333333,0.333333,0.053571,0.000000,0.0,0 <= ... < 200 DM,existing credits paid back duly till now,...,... < 100 DM,1 <= ... < 4 years,female : divorced/separated/married,none,real estate,none,own,skilled employee / official,none,yes
2,N,0.117647,0.101574,0.333333,0.666667,0.535714,0.000000,1.0,no checking account,critical account/ other credits existing (not ...,...,... < 100 DM,4 <= ... < 7 years,male : single,none,real estate,none,own,unskilled - resident,none,yes
3,N,0.558824,0.419941,0.333333,1.000000,0.464286,0.000000,1.0,< 0 DM,existing credits paid back duly till now,...,... < 100 DM,4 <= ... < 7 years,male : single,guarantor,if not A121 : building society savings agreeme...,none,for free,skilled employee / official,none,yes
4,Y,0.294118,0.254209,0.666667,1.000000,0.607143,0.333333,1.0,< 0 DM,delay in paying off in the past,...,... < 100 DM,1 <= ... < 4 years,male : single,none,unknown / no property,none,for free,skilled employee / official,none,yes


In [13]:
# enconded_df, encoder_dict = label_encode(scaled_df, categorical_cols)

In [14]:
from sklearn.preprocessing import LabelEncoder

target_label_encoder = LabelEncoder()
dummy_df[target_name] = target_label_encoder.fit_transform(dummy_df[target_name])

In [15]:
train_df, test_df = train_test_split(dummy_df, train_size=.8, random_state=seed, shuffle=True)

In [16]:
X_train = np.array(train_df[ohe_feature_names])
y_train = np.array(train_df[target_name])
X_test = np.array(test_df[ohe_feature_names])
y_test = np.array(test_df[target_name])

In [17]:
### Train
nn = model= tf.keras.models.Sequential(
            [
                tf.keras.layers.Dense(24,activation='relu'),
                tf.keras.layers.Dense(12,activation='relu'),
                tf.keras.layers.Dense(12,activation='relu'),
                tf.keras.layers.Dense(12,activation='relu'),
                tf.keras.layers.Dense(12,activation='relu'),
                tf.keras.layers.Dense(1),
                tf.keras.layers.Activation(tf.nn.sigmoid),
            ]
        )
nn.compile(optimizer="Adam", loss='binary_crossentropy', metrics=['accuracy'])
nn.fit(X_train, y_train, batch_size=64, epochs=20, shuffle=True)

models = {
    "dt": DecisionTreeClassifier().fit(X_train,y_train),
    "rfc": RandomForestClassifier().fit(X_train,y_train),
    "nn": nn,
}

pickle.dump(models['dt'], open('./saved_models/dt.p', 'wb'))
pickle.dump(models['rfc'], open('./saved_models/rfc.p', 'wb'))
models['nn'].save('./saved_models/nn.h5',overwrite=True)


Epoch 1/20
13/13 [==============================] - 0s 695us/step - loss: 0.6778 - accuracy: 0.6369
Epoch 2/20
13/13 [==============================] - 0s 607us/step - loss: 0.6382 - accuracy: 0.6824
Epoch 3/20
13/13 [==============================] - 0s 650us/step - loss: 0.5835 - accuracy: 0.7285
Epoch 4/20
13/13 [==============================] - 0s 502us/step - loss: 0.5937 - accuracy: 0.6946
Epoch 5/20
13/13 [==============================] - 0s 487us/step - loss: 0.5590 - accuracy: 0.7149
Epoch 6/20
13/13 [==============================] - 0s 504us/step - loss: 0.5548 - accuracy: 0.7027
Epoch 7/20
13/13 [==============================] - 0s 488us/step - loss: 0.5353 - accuracy: 0.7194
Epoch 8/20
13/13 [==============================] - 0s 493us/step - loss: 0.5150 - accuracy: 0.7326
Epoch 9/20
13/13 [==============================] - 0s 468us/step - loss: 0.5110 - accuracy: 0.7354
Epoch 10/20
13/13 [==============================] - 0s 466us/step - loss: 0.4732 - accuracy: 0.7612

In [18]:
### Load
models = {}
models['dt'] = pickle.load(open('./saved_models/dt.p', 'rb'))
models['rfc'] = pickle.load(open('./saved_models/rfc.p', 'rb'))
models['nn'] = tf.keras.models.load_model('./saved_models/nn.h5')

## Initialise NN output shape as (None, 1) for tensorflow.v1
models['nn'].predict(np.zeros((2, X_train.shape[-1])))


array([[0.5436064],
       [0.5436064]], dtype=float32)

In [19]:
example_data = X_test[0, :].reshape(1,-1)

dt_pred = models['dt'].predict(example_data)[0]
rfc_pred = models['rfc'].predict(example_data)[0]
nn_pred = models['nn'].predict(example_data)[0][0]

print(f"DT [{dt_pred}], RFC [{rfc_pred}], NN [{nn_pred}]")

DT [1], RFC [1], NN [0.716882050037384]


# DiCE

In [20]:
import dice_ml

In [21]:
d = dice_ml.Data(dataframe=scaled_df, continuous_features=numerical_cols, outcome_name=target_name)

In [22]:
# m = dice_ml.Model(model=models['dt'], backend="sklearn")
# exp = dice_ml.Dice(d,m)

In [29]:
class RecordWrapper():
    def __init__(self, model, cat_to_ohe_cat, ohe_feature_names):
        self.all_inputs = []
        self.model = model
        self.cat_to_ohe_cat = cat_to_ohe_cat
        self.ohe_feature_names = ohe_feature_names


    def dice_to_input(self, input_df):
        x = input_df.copy(deep=True)

        for k in cat_to_ohe_cat.keys():
            for ohe_col in cat_to_ohe_cat[k]:
                x[ohe_col] = x[k].apply(lambda v: 1 if v in ohe_col else 0) 
            x.drop([k], axis=1, inplace=True)
            
        return np.array(x[ohe_feature_names])

    def predict_proba(self, x):
        self.all_inputs.append(x)
        cf_input = self.dice_to_input(x)
        return self.model.predict_proba(cf_input)

    def predict(self, x):
        self.all_inputs.append(x)
        cf_input = self.dice_to_input(x)
        return self.model.predict(cf_input)

class NNRecordWrapper():
    def __init__(self, model, cat_to_ohe_cat, ohe_feature_names):
        self.all_inputs = []
        self.model = model
        self.cat_to_ohe_cat = cat_to_ohe_cat
        self.ohe_feature_names = ohe_feature_names


    def dice_to_input(self, input_df):
        x = input_df.copy(deep=True)

        for k in cat_to_ohe_cat.keys():
            for ohe_col in cat_to_ohe_cat[k]:
                x[ohe_col] = x[k].apply(lambda v: 1 if v in ohe_col else 0) 
            x.drop([k], axis=1, inplace=True)
            
        return np.array(x[ohe_feature_names])

    def predict(self, x):
        self.all_inputs.append(x)
        cf_input = self.dice_to_input(x)
        return self.model.predict(tf.constant(cf_input.astype(float)))

    def predict_proba(self, x):
        self.all_inputs.append(x)
        cf_input = self.dice_to_input(x)
        return self.model.predict(tf.constant(cf_input.astype(float)))
        

In [30]:
all_wrapped_models = {
    'dt': RecordWrapper(models['dt'], cat_to_ohe_cat, ohe_feature_names),
    'rfc': RecordWrapper(models['rfc'], cat_to_ohe_cat, ohe_feature_names),
    'nn': NNRecordWrapper(models['nn'], cat_to_ohe_cat, ohe_feature_names),
}

In [31]:
# dt_record = RecordWrapper(models['dt'], cat_to_ohe_cat, ohe_feature_names)
# m = dice_ml.Model(model=dt_record, backend="sklearn")
# exp = dice_ml.Dice(d,m)
# dice_exp = exp.generate_counterfactuals(scaled_df.iloc[1:2], total_CFs=5, desired_class="opposite")
# dice_exp.cf_examples_list[0].final_cfs_df.iloc[0][:-1]

In [32]:
dice_cfs = {
    'dt': dice_ml.Dice(d,dice_ml.Model(model=all_wrapped_models['dt'], backend="sklearn")),
    'rfc': dice_ml.Dice(d,dice_ml.Model(model=all_wrapped_models['rfc'], backend="sklearn")),
    'nn': dice_ml.Dice(d,dice_ml.Model(model=all_wrapped_models['nn'], backend="sklearn"))
}

In [33]:
num_instances = 5
num_cf_per_instance = 1

In [34]:
results = {}

for k in dice_cfs.keys():
    results[k] = []
    print(f"Finding counterfactual for {k}")
    for idx, instance in enumerate(scaled_df.iloc[test_df[0:num_instances].index].iloc):
        print(f"instance {idx}")
        for num_cf in range(num_cf_per_instance):
            print(f"CF {num_cf}")
            start_t = time()

            input_query = pd.DataFrame([instance.to_dict()])
            ground_truth = input_query[target_name][0]
            exp = dice_cfs[k].generate_counterfactuals(input_query, total_CFs=1, sample_size=200, desired_class="opposite")

            # dice_exp = dice_cfs['nn'].generate_counterfactuals(scaled_df.iloc[1:2], total_CFs=1, desired_class="opposite")
            # dice_exp.cf_examples_list[0].final_cfs_df.iloc[0][:-1]

            if k=='nn':
                prediction = target_label_encoder.inverse_transform((all_wrapped_models[k].predict(input_query)[0]> 0.5).astype(int))[0]
            else:
                prediction = target_label_encoder.inverse_transform(all_wrapped_models[k].predict(input_query))[0]
            
            end_t = time ()
            running_time = end_t - start_t
            results[k].append({
                "input": input_query,
                "cf": exp.cf_examples_list[0].final_cfs_df,
                "running_time": running_time,
                "ground_truth": ground_truth,
                "prediction": prediction,
            })

  0%|          | 0/1 [00:00<?, ?it/s]

Finding counterfactual for dt
instance 0
CF 0
instance 1
CF 0


  0%|          | 0/1 [00:00<?, ?it/s]

instance 2
CF 0
instance 3
CF 0


  0%|          | 0/1 [00:00<?, ?it/s]

...

No Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 00 sec
instance 3
CF 0


  0%|          | 0/1 [00:00<?, ?it/s]

No Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 00 sec
instance 4
CF 0


100%|██████████| 1/1 [00:00<00:00,  1.42it/s]

No Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 00 sec


In [ ]:
# scaled_input_df = results['dt'][0]['input'].copy(deep=True)
# origin_columns = [f"origin_input_{col}"  for col in scaled_input_df.columns]
# origin_input_df = scaled_input_df.copy(deep=True)
# scaled_input_df.columns = [f"scaled_input_{col}"  for col in scaled_input_df.columns]

# origin_input_df[numerical_cols] = scaler.inverse_transform(origin_input_df[numerical_cols])
# origin_input_df.columns = origin_columns

# scaled_cf_df = results['dt'][0]['cf'].copy(deep=True)
# scaled_cf_df.loc[0, target_name] = target_label_encoder.inverse_transform([scaled_cf_df.loc[0, target_name]])[0]
# origin_cf_columns = [f"origin_cf_{col}"  for col in scaled_cf_df.columns]
# origin_cf_df = scaled_cf_df.copy(deep=True)
# scaled_cf_df.columns = [f"scaled_cf_{col}"  for col in scaled_cf_df.columns]

# origin_cf_df[numerical_cols] = scaler.inverse_transform(origin_cf_df[numerical_cols])
# origin_cf_df.columns = origin_cf_columns

# final_df = pd.DataFrame([{}])
# final_df = final_df.join([scaled_input_df, origin_input_df, scaled_cf_df, origin_cf_df])
# final_df['running_time'] = results['dt'][0]['running_time']
# final_df['Found'] = "Y" if not results['dt'][0]['cf'] is None else "N"


In [35]:
all_df = {}

for k in results.keys():

    all_data = []

    for i in range(len(results[k])):
        final_df = pd.DataFrame([{}])

        scaled_input_df = results[k][i]['input'].copy(deep=True)
        origin_columns = [f"origin_input_{col}"  for col in scaled_input_df.columns]
        origin_input_df = scaled_input_df.copy(deep=True)
        scaled_input_df.columns = [f"scaled_input_{col}"  for col in scaled_input_df.columns]

        origin_input_df[numerical_cols] = scaler.inverse_transform(origin_input_df[numerical_cols])
        origin_input_df.columns = origin_columns

        final_df = final_df.join([scaled_input_df, origin_input_df])

        if not results[k][i]['cf'] is None:
            scaled_cf_df = results[k][i]['cf'].copy(deep=True)
            scaled_cf_df.loc[0, target_name] = target_label_encoder.inverse_transform([scaled_cf_df.loc[0, target_name]])[0]
            origin_cf_columns = [f"origin_cf_{col}"  for col in scaled_cf_df.columns]
            origin_cf_df = scaled_cf_df.copy(deep=True)
            scaled_cf_df.columns = [f"scaled_cf_{col}"  for col in scaled_cf_df.columns]

            origin_cf_df[numerical_cols] = scaler.inverse_transform(origin_cf_df[numerical_cols])
            origin_cf_df.columns = origin_cf_columns

            final_df = final_df.join([scaled_cf_df, origin_cf_df])

        # final_df = final_df.join([scaled_input_df, origin_input_df, scaled_cf_df, origin_cf_df])
        final_df['running_time'] = results[k][i]['running_time']
        final_df['Found'] = "Y" if not results[k][i]['cf'] is None else "N"
        final_df['ground_truth'] = results[k][i]['ground_truth'] 
        final_df['prediction'] = results[k][i]['prediction'] 

        all_data.append(final_df)

    all_df[k] = pd.concat(all_data)

In [36]:
for df_k in all_df.keys():
    all_df[df_k].to_csv(f"./results/dice_german_{df_k}_result.csv")